# Good tutorial on flask & server
https://pythonise.com/series/learning-flask/rendering-html-files-with-flask

In [ ]:
# A very simple Flask Hello World app for you to get started with...
from flask import Flask, render_template, request, url_for, jsonify
import requests
import os


# IT IS VERY IMPORTANT TO SPECIFY THE ABSOLUTE (STARTING WITH FORWARD SLASH (I.E. FROM THE ROOT)) OR 
# RELATIVE PATHS TO WHERE THE STATIC (CSS, JS) AND TEMPLATE (HTML) FILES WILL BE. IT'S OKAY TO HAVE THEM IN 
# THE SAME FOLDER
app = Flask(__name__,
            static_url_path='', 
            static_folder='/Users/romankoshkin/ridesmart',
            template_folder='/Users/romankoshkin/ridesmart')

@app.route('/site')
def site():
    return render_template("index.html")

@app.route('/')
def hello_world():
    return 'Hello from Flask!'

@app.route('/order', methods=['POST'])
def my_test_endpoint():
    input_json = request.get_json(force=True) # force=True, if another developer forgot to set the MIME type to 'application/json'
    print([(k, input_json[k]) for k in input_json.keys()])
    notify(input_json)
    add2DB(input_json)
    dictToReturn = {'answer':42}
    return jsonify(dictToReturn)

@app.route('/query-example')
def query_example():
    language = request.args.get('language') #if key doesn't exist, returns None
    framework = request.args['framework']   #if key doesn't exist, returns a 400, bad request error
    website = request.args.get('website')

    return '''<h1>The language value is: {}</h1><h1>The framework value is: {}</h1><h1>The website value is: {}</h1>'''.format(language, framework, website)

def add2DB(input_json):
    line = ''
    cwd = os.getcwd()
    with open(cwd + '/DB.txt', 'a') as f:
        for k in input_json.keys():
            line += str(input_json[k]) + ','
        line += '\n'
        f.writelines(line)

def notify(input_json):
    bot_token = '1103695667:AAF6_WLEpnh9b4rminEKMzSnje7BiRUPDWQ'
    bot_chatID = '348368436'
    txt = [input_json[k] for k in input_json.keys()]
    bot_message = 'city: {}, Country: {}, destination: {}, desired_GMT: {}, time_sent: {}, appID: {}'.format(*txt)
    send_text = ('https://api.telegram.org/bot' + bot_token +
             '/sendMessage?chat_id=' + bot_chatID +
             '&parse_mode=Markdown&text=' + bot_message)
    response = requests.get(send_text)
    response.json()

def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

@app.route('/shutdown', methods=['POST'])
def shutdown():
    shutdown_server()
    return 'Server shutting down...'


if __name__ == '__main__':
#     app.run(debug=False, port=5000) #run app in debug mode on port 5000
    app.run(debug=False) #run app in debug mode on port 5000

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


[('city', 'Moscow'), ('country', 'RU'), ('destination', 'Бирюлёво'), ('desired_GMT', 17), ('time_sent', 17), ('appID', 7561)]


127.0.0.1 - - [04/Aug/2020 12:27:15] "POST /order HTTP/1.0" 200 -
